# Get League Table and Stats

In [157]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import numpy as np

In [158]:
# URL of the website
url = "https://fbref.com/en/comps/10/Championship-Stats"

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful (status code 200)
    # Parse the HTML content of the page
soup = BeautifulSoup(response.text, "html.parser")

    # Find the table using its HTML class
table = soup.find("table", {"class": "stats_table"})

    # Extract table data into a list of lists
table_data = []
for row in table.find_all("tr"):
    row_data = [cell.text.strip() for cell in row.find_all(["th", "td"])]
    table_data.append(row_data)

    # Create a DataFrame from the list of lists
columns = table_data[0]  # Assuming the first row contains column headers
data = table_data[1:]
league_table_df = pd.DataFrame(data, columns=columns)

# Get Fixtures 

In [159]:
# URL of the website
url = "https://fbref.com/en/comps/10/schedule/Championship-Scores-and-Fixtures"

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.text, "html.parser")

    # Find the table containing fixtures
    table = soup.find("table", {"class": "stats_table"})

    # Extract data into a list of dictionaries
    fixtures_data = []
    for row in table.find_all("tr")[1:]:  # Skip the header row
        columns = row.find_all(["th", "td"])
        
         # Extracting specific columns based on the structure of the table
        fixture_type = columns[0].text.strip()
        weekday = columns[1].text.strip()
        date = columns[2].text.strip()
        time = columns[3].text.strip()
        home_team = columns[4].text.strip()
        home_xG = columns[5].text.strip()
        result = columns[6].text.strip()
        away_xG = columns[7].text.strip()
        away_team = columns[8].text.strip()
        attendance = columns[9].text.strip()
        stadium = columns[10].text.strip()
        referee = columns[11].text.strip()

        fixture_info = {
            "fixture_type": fixture_type,
            "weekday": weekday,
            "date": date,
            "time": time,
            "home_team": home_team,
            "home_xG": home_xG,
            "result": result,
            "away_xG": away_xG,
            "away_team": away_team,
            "attendance": attendance,
            "stadium": stadium,
            "referee": referee
        }

        fixtures_data.append(fixture_info)

    # Create a DataFrame from the list of dictionaries
    fixtures_table = pd.DataFrame(fixtures_data)

In [160]:
fixtures_table.head()
#need to fix from attendance on

,fixture_type,weekday,date,time,home_team,home_xG,result,away_xG,away_team,attendance,stadium,referee
0,1,Fri,2023-08-04,20:00,Sheffield Weds,0.5,1–2,1.4,Southampton,"28,558",Hillsborough Stadium,Robert Madley
1,1,Sat,2023-08-05,15:00,Bristol City,0.9,1–1,1.3,Preston,"29,359",Ashton Gate Stadium,David Webb
2,1,Sat,2023-08-05,15:00,Plymouth Argyle,2.4,3–1,2.0,Huddersfield,"16,446",Home Park,Matt Donohue
3,1,Sat,2023-08-05,15:00,Stoke City,2.4,4–1,0.9,Rotherham Utd,"22,601",Bet365 Stadium,John Busby
4,1,Sat,2023-08-05,15:00,Middlesbrough,0.8,0–1,1.2,Millwall,"29,359",Riverside Stadium,Gavin Ward


In [161]:
fixtures_table['is_game_complete'] = np.where(len(fixtures_table['result']) > 0, 1, 0)
#Checking to see if game is complete... if the result column is populated at all, it's been played

# Getting Game IDs

In [162]:
# URL of the website
url = "https://fbref.com/en/comps/10/schedule/Championship-Scores-and-Fixtures"

# Parse the HTML content of the page
soup = BeautifulSoup(response.text, 'html.parser')

# Find all <td> elements with data-stat="score"
score_cells = soup.find_all('td', {'data-stat': 'score'})

# Extract game IDs from the <a> tags within the score_cells, the third element is the game ID
game_ids = [cell.find('a')['href'].split('/')[3] for cell in score_cells if cell.find('a')]
# Then strip the white space in the IDs
game_ids = [x.strip(' ') for x in game_ids]

In [163]:
game_ids = game_ids[0:4]
#just use three to test

# Scraping Shot Data

In [164]:
all_data = []

for unique_id in game_ids:
    # URL of the page to scrape
    url = f'https://fbref.com/en/matches/{unique_id}/'

    # Get the game ID from the URL
    game_id = url.split("/")[-2]

    # Send a GET request to the URL
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    # Find the Shots table on the page
    shots_table = soup.find("div", {"id": "switcher_shots"})

    # Create empty lists to store data and create the columns
    data = []
    headers = ['game_id', 'minute', 'player', 'team', 'xG', 'psxG', 'result', 'distance', 'body_part', 'notes', 'sca_1_player', 'event_1', 'sca_2_player', 'event_2']

    # Extract table rows
    rows = shots_table.find("tbody").find_all("tr")
    for row in rows:
        row_data = [game_id] + [cell.get_text().strip() for cell in row.find_all(["th", "td"])]
        data.append(row_data)

    # Find the home and away teams and formations
    team_headers = soup.find_all("th", {"colspan": "2"})
    teams = [header.text.strip().split(" (")[0] for header in team_headers if "(" in header.text.strip()]
    formations = [header.text.strip().split(" (")[1][:-1] for header in team_headers if "(" in header.text.strip()]

    # Extracting the home and away teams and formations
    home_team = teams[0]
    away_team = teams[1]
    home_formation = formations[0]
    away_formation = formations[1]

    # Find the location and match date
    location = soup.find("div", {"class": "scorebox_meta"}).find_all("small")[3].text
    match_date = soup.find("span", class_="venuetime").get("data-venue-date", None)

    # Append additional columns to headers
    headers.extend(['home_team', 'away_team', 'home_formation', 'away_formation', 'location', 'match_date'])

    # Append data to all_data with additional columns
    for row in data:
        row.extend([home_team, away_team, home_formation, away_formation, location, match_date])
        all_data.append(row)
        
# Create a DataFrame from the scraped data
shots_df = pd.DataFrame(all_data, columns=headers)

In [165]:
# Add 
shots_df['is_home_shot'] = np.where(shots_df['team'] == shots_df['home_team'], 1, 0)
shots_df['is_away_shot'] = np.where(shots_df['team'] ==  shots_df['away_team'], 1, 0)
shots_df['location'] = location
shots_df['match_date'] = match_date
shots_df['competition'] = 'English Championship'

In [166]:
# Define a function to calculate total score for each game
def calculate_total_score(group, shot):
    total_score = 0
    total_scores = []

    # Flag to indicate whether the current row is after a 'Goal'
    after_goal = False

    for idx, (result, is_shot) in enumerate(zip(group, shot)):
        # Check if the current row is after a 'Goal'
        if after_goal:
            # Increment the total score
            total_score += 1
            # Reset the flag
            after_goal = False

        # Check if the current result is 'Goal' and the corresponding shot is 1
        if result == 'Goal' and is_shot == 1:
            # Set the flag to True for the next iteration
            after_goal = True
        
        # Append the current total score
        total_scores.append(total_score)

    return total_scores

# Group by game_id and apply the function to calculate total score for home team
shots_df['home_score'] = shots_df.groupby('game_id')['result'].transform(lambda x: calculate_total_score(x, shots_df['is_home_shot']))

# Group by game_id and apply the function to calculate total score for away team
shots_df['away_score'] = shots_df.groupby('game_id')['result'].transform(lambda x: calculate_total_score(x, shots_df['is_away_shot']))


In [167]:
#Creating a nil nil column using numpy where and logical_and operators
shots_df['is_nil_nil'] = np.where(np.logical_and(shots_df['home_score'] == 0, shots_df['away_score'] == 0), 1, 0)

In [169]:
#Adding opposing team -- Whenever it's a home shot we want the away team, otherwise (away shot) we want the home team
shots_df['opposing_team'] = np.where(shots_df['is_home_shot'] == 1, shots_df['away_team'], shots_df['home_team'])

# Adding xG while nil-nil to league_table_df

In [172]:
league_table_df

,Rk,Squad,MP,W,D,L,GF,GA,GD,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes
0,1,Leeds United,38,25,7,6,70,28,+42,82,2.16,67.2,31.4,+35.8,+0.94,W D W W W,"35,838",Crysencio Summerville - 15,Illan Meslier,
1,2,Leicester City,37,26,4,7,74,33,+41,82,2.22,67.5,35.1,+32.4,+0.88,L L L W D,"31,053",Jamie Vardy - 13,Mads Hermansen,
2,3,Ipswich Town,38,24,9,5,80,49,+31,81,2.13,63.5,39.3,+24.3,+0.64,W W W L W,"28,825","Conor Chaplin, Nathan Broadhead - 12",Václav Hladký,
3,4,Southampton,36,22,7,7,73,47,+26,73,2.03,64.3,37.6,+26.6,+0.74,W L L W W,"29,333",Adam Armstrong - 18,Gavin Bazunu,
4,5,West Brom,38,19,9,10,59,36,+23,66,1.74,46.3,39.5,+6.8,+0.18,D W D W W,"23,956",Brandon Thomas-Asante - 9,Alex Palmer,
5,6,Norwich City,38,18,7,13,69,54,+15,61,1.61,52.7,52.5,+0.2,0.00,D W L W W,"25,970",Josh Sargent - 13,Angus Gunn,
6,7,Hull City,37,16,10,11,53,46,+7,58,1.57,45.1,39.1,+6.0,+0.16,W D D D D,"21,762","Ozan Tufan, Aaron Connolly... - 8",Ryan Allsop,
7,8,Coventry City,37,15,12,10,59,43,+16,57,1.54,53.7,41.8,+11.9,+0.32,W L L W W,"24,801",Haji Wright - 13,Bradley Collins,
8,9,Preston,37,16,8,13,49,54,-5,56,1.51,33.0,45.8,-12.8,-0.35,D W D L W,"16,759",Will Keane - 12,Freddie Woodman,
9,10,Middlesbrough,38,16,6,16,53,52,+1,54,1.42,56.2,41.9,+14.3,+0.38,L W W W D,"26,764","Emmanuel Latte Lath, Marcus Forss - 7",Seny Dieng,


# To Do

1a. Fix numpy where columns 
1. EDA
2. Start Modeling